In [120]:
import requests
from selenium import webdriver
from webdriver_manager.utils import chrome_version
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import pandas as pd
# これをimportしておくことで自分でcheromedriverをインストールする必要がない
import chromedriver_binary
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [58]:
class SeleniumSetUp:
    def __init__(self, home_url:str, user_id = "none", password="none"):
        self.op = Options()
        # --headlessだけではOSによって動かない、プロキシが弾かれる、
        # CUI用の省略されたHTMLが帰ってくるなどの障害が出ます。
        # 長いですが、これら6行あって最強かつどんな環境でも動きますので、必ず抜かさないようにしてください。
        self.op.add_argument("no-sandbox")
        self.op.add_argument("--disable-extensions")
        # self.op.add_argument("--headless")
        self.op.add_argument('--disable-gpu')  
        self.op.add_argument('--ignore-certificate-errors')
        self.op.add_argument('--allow-running-insecure-content')
        self.op.add_argument('--disable-web-security')
        self.op.add_argument('--disable-desktop-notifications')
        self.op.add_argument("--disable-extensions")
        self.op.add_argument('--lang=ja')
#         self.op.add_argument('--blink-settings=imagesEnabled=false')
        self.op.add_argument('--disable-dev-shm-usage')
        self.op.add_argument('--proxy-server="direct://"')
        self.op.add_argument('--proxy-bypass-list=*')    
        self.op.add_argument('--start-maximized')

        self.home_url = home_url

        self.user_id = user_id
        self.password = password  

    def driver_set(self):
        driver = webdriver.Chrome(chrome_options = self.op)
        driver.get(self.home_url)
        time.sleep(1)
        return driver

    def login(self, input_id, input_pass, login_button):
        # ユーザーidのinput要素を取得する
        input_name.send_keys(self.user_id)

        #  パスワードのinput要素を取得する
        input_pass.send_keys(self.password)

        #  ログインボタンの要素を取得する
        login_button.click()

        time.sleep(1)
        return driver

    #取得したデータを統合
    def auto_csv(self, datas: list, cols: list, file_name: str, make=False):
        if len(datas) != len(cols):
            return "希望のデータのカラムと作成したデータのカラムの数が異なります"

        else:
            pd_datas = []

            for data in datas:
                pd_datas.append(pd.DataFrame(data))

            concat = pd.concat([li.T for li in pd_datas]).T
            concat.columns = cols

            if make == True:
                concat.to_csv("{}.csv".format(file_name))
            else:
                pass

            return concat

    # 元のタブを閉じて２つ目のタブをメインタブにする
    def tab_move(self, driver, close=True):
        # tab移動
        # driver.window_handles[タブの番号]
        driver.switch_to.window(driver.window_handles[1])
        driver.switch_to.window(driver.window_handles[0])
        if close == True:

            # 初めに開いていたウインドウを閉じる
            driver.close()

            # 新しく開かれたタブを新規にインスタンスウインドウとする これをしないとNosuchElementエラーになる
            driver.switch_to.window(driver.window_handles[0])

        else:
            driver.switch_to.window(driver.window_handles[1])

        return driver

    # 元のタブを閉じて２つ目のタブをメインタブにする
    def tab_return(self, driver, close=True):
        # tab移動
        # driver.window_handles[タブの番号]
        driver.switch_to.window(driver.window_handles[0])
        driver.switch_to.window(driver.window_handles[1])
        if close == True:
            driver.switch_to.window(driver.window_handles[0])
            # 初めに開いていたウインドウを閉じる
            driver.close()

            # 新しく開かれたタブを新規にインスタンスウインドウとする これをしないとNosuchElementエラーになる
            driver.switch_to.window(driver.window_handles[0])

        else:
            driver.switch_to.window(driver.window_handles[0])

        return driver

In [190]:
HOME = "https://ai-products.net/category_page/chatbot/"

COL_ELEMENTS = [
                                "title",
                                "company",
                                "abst1",
                                "abst2",
                                "price",
                                "initial_price",
                                "free_plan",
                                "free_trial",
                                "detail_link"
                            ]

COL_DETAILS =    [
                                "detail_link",
                                "point1",
                                "point2",
                                "point3",
                                "target_ind",
                                "goal",
                                "scale"
                                ]

# チャットボットサービスのサービス比較と企業一覧

In [60]:
setup = SeleniumSetUp(HOME)

In [61]:
driver = setup.driver_set()

/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:29: DeprecationWarning: use options instead of chrome_options


In [62]:
# すべての要素が入ったリンク
elements = driver.find_elements_by_class_name("serviceGroup")

# 対応リンク
title = elements[0].find_element_by_class_name("service").text
campany = elements[0].find_element_by_class_name("corporateName").text
abst1 = elements[0].find_elements_by_tag_name("p")[1].text
abst2 = elements[0].find_elements_by_tag_name("p")[2].text
price = elements[0].find_element_by_xpath("//dl[@class='cost']/dd").text
initial_price = elements[0].find_element_by_xpath("//dl[@class='initial-cost']/dd").text
free_plan = elements[0].find_element_by_xpath("//dl[@class='free-plan']/dd").text
free_trial = elements[0].find_element_by_xpath("//dl[@class='free-trial']/dd").text
detail_link = elements[0].find_element_by_class_name("detail-link").get_attribute("href")



In [102]:
title_   = []
com_ = []
abs1_ = []
abs2_ = []
pri_    = []
ini_     = []
fpl_    = []
ftr_    = []
dli_    = []




for i in range(len(elements)):
    title = elements[i].find_element_by_class_name("service").text
    company = elements[i].find_element_by_class_name("corporateName").text
    abst1 = elements[i].find_elements_by_tag_name("p")[1].text
    abst2 = elements[i].find_elements_by_tag_name("p")[2].text
    price = elements[i].find_element_by_class_name("cost").text.replace("利用料金\n","")
    initial_price = elements[i].find_element_by_class_name("initial-cost").text.replace("初期費用\n","")
    free_plan = elements[i].find_element_by_class_name("free-plan").text.replace("無料プラン\n", "")
    free_trial = elements[i].find_element_by_class_name("free-trial").text.replace("無料トライアル\n", "")
    detail_link = elements[i].find_element_by_class_name("detail-link").get_attribute("href")
    
    title_.append(title)
    com_.append(company)
    abs1_.append(abst1)
    abs2_.append(abst2)
    pri_.append(price)
    ini_.append(initial_price)
    fpl_.append(free_plan)
    ftr_.append(free_trial)
    dli_.append(detail_link)
    print(f"{i+1} / {len(elements)}を実行中")
    

1 / 39を実行中
2 / 39を実行中
3 / 39を実行中
4 / 39を実行中
5 / 39を実行中
6 / 39を実行中
7 / 39を実行中
8 / 39を実行中
9 / 39を実行中
10 / 39を実行中
11 / 39を実行中
12 / 39を実行中
13 / 39を実行中
14 / 39を実行中
15 / 39を実行中
16 / 39を実行中
17 / 39を実行中
18 / 39を実行中
19 / 39を実行中
20 / 39を実行中
21 / 39を実行中
22 / 39を実行中
23 / 39を実行中
24 / 39を実行中
25 / 39を実行中
26 / 39を実行中
27 / 39を実行中
28 / 39を実行中
29 / 39を実行中
30 / 39を実行中
31 / 39を実行中
32 / 39を実行中
33 / 39を実行中
34 / 39を実行中
35 / 39を実行中
36 / 39を実行中
37 / 39を実行中
38 / 39を実行中
39 / 39を実行中


In [104]:
item_pack = [
    title_,
    com_,
    abs1_,
    abs2_,
    pri_,
    ini_,
    fpl_,
    ftr_,
    dli_
]

In [107]:
item_pack[4]

['月額10万円～',
 '月額10万円～',
 'お問い合わせください',
 '20,000円/月〜',
 'お問い合わせください',
 'お問い合せください',
 '個別見積となりますので、\nお問い合わせください。',
 '135,000円/月〜',
 '月額50,000円～',
 '98,000円',
 'お問い合わせください',
 '月額1万円〜',
 '1500円～',
 '6,980円/月',
 'お問い合わせください',
 '50,000円/月〜',
 '月額5万円',
 'お問い合わせください',
 '初期費用の20%',
 '100,000円～',
 'お問い合わせ下さい',
 'お問い合わせ下さい',
 '30,000円/月～',
 '0円',
 '70,000円/月',
 '10万円～',
 '月額2,980円～',
 '20万円～',
 '30万円',
 'お問い合わせください',
 '30万円～（月額）',
 '月額\u30006,000円/1ユーザー',
 '12,000円/1ID',
 'お問い合わせ下さい',
 '7万円～30万円',
 '月額5000円～',
 '120,000円',
 '月額1万円～',
 'お問い合わせ下さい']

In [108]:
def check_data_len(check_item:list) -> list:
    
    # １つ目のcheck_itemの個数をbase_lineとして比較する
    base_line = len(check_item[0])
    all_len = [base_line]
    check = True
    
    for i in range(len(check_item)-1):
        
        if len(check_item[i+1]) != base_line:
            print(f"番号[{i}] alert: 前の要素 {base_line}と今回の要素 {len(check_item[i+1])}で個数が合いません")
            check = False

        
        # 最後はbase_lineを更新して要素の数を格納するリストを作成する
        base_line = len(check_item[i+1])
        all_len.append(base_line)
        
    if check:
        print("all elements is same !")
    else:
        print("error")
        
    return all_len

In [109]:
all_l = check_data_len(item_pack)

all elements is same !


In [116]:
concat_ = setup.auto_csv(item_pack, COL_ELEMENTS, "elements", make=True)

In [117]:
concat_

,title,company,abst1,abst2,price,initial_price,free_plan,free_trial,detail_link
0,WisTalk（ウィズトーク）,パナソニック ソリューションテクノロジー株式会社,問い合わせ対応業務の負荷を約50％軽減！\n「Microsoft Teams」や「LINE」...,パナソニック独自開発のAIエンジンを搭載した「WisTalk」が機能強化して登場！\n「未知...,月額10万円～,0円,なし,あり,https://ai-products.net/product/wistalk/
1,HUE Works Suite DX Solutions Chatbot,株式会社ワークスアプリケーションズ,AI研究の専門機関”徳島人工知能NLP研究所”が開発！\n日本最大規模の辞書を搭載したAIチ...,社内外の問い合わせ業務を効率化するAI対話ツールです。\nNLP（自然言語処理）技術を応用す...,月額10万円～,30万円～,なし,あり,https://ai-products.net/product/hue/
2,amie Helpbot,ペネトレイト・オブ・リミット株式会社,シナリオ作成は一切不要！\n既にあるドキュメントやＦＡＱを自動で学習！\n最短1日で導入可能...,既存のマニュアルを自動で学習しわずか2ステップでチャットボットが作成できます。\nチャットボ...,お問い合わせください,お問い合わせください,なし,あり,https://ai-products.net/product/amie-helpbot/
3,AURA-アウラ,Micoworks株式会社,【継続利用率：98.8%】\nLINEで本格的なマーケティングを行うなら、\nLINEツール...,AURAはLINEを使って、煩雑になりがちな顧客管理やマーケティングを効率化し、数字の最適化...,"20,000円/月〜",お問い合わせください,お問い合わせください,お問い合わせください,https://ai-products.net/product/aura/
4,Vareal NLP（自然言語処理）システム構築/導入サービス,Vareal株式会社,Vareal NLP（自然言語処理）システム構築/導入サービス,お客様のニーズに応じたNLP（自然言語処理）を用いたシステムのコンサルティング、\nAIモデ...,お問い合わせください,お問い合わせください,お問い合わせください,お問い合わせください,https://ai-products.net/product/vareal-nlp/
5,AIチャット SCHATTI,株式会社ZeQ,対応効率35％UP↑ 返答時間20％DOWN↓\n学習機能を備えたAI回答補助&チャットボッ...,SCHATTI(スチャッティ)は、日々の顧客対応で学習したデータよりAIが最適な回答を提示す...,お問い合せください,お問い合せください,なし,あり,https://ai-products.net/product/schatti/
6,Benefitter,伊藤忠テクノソリューションズ株式会社,CTCの最適解！働き方を進化させる\nDXチャットボットプラットフォーム,Benefitterは、高度なAIボットを短期間で作成できるチャットボット開発プラットフォー...,個別見積となりますので、\nお問い合わせください。,個別見積となりますので、\nお問い合わせください。,なし,あり,https://ai-products.net/product/benefitter/
7,デジタルヒューマン,デジタルヒューマン株式会社,人間らしさとAIのメリットを融合。\n新しい顧客体験を提供する\n「対話+感情表現 AIイン...,デジタルヒューマン（Digital Humans）はチャットボットやNLPと連携し、\n対話...,"135,000円/月〜",0円,なし,あり,https://ai-products.net/product/digital-human/
8,hachidori,hachidori株式会社,hachidori,hachidoriは、圧倒的な開発実績とNoCodeで開発可能なユーザビリティを備えた国産初...,"月額50,000円～","100,000円～",なし,なし,https://ai-products.net/product/hachidori/
9,kantan chat,マスターピース・グループ株式会社,最短3ヶ月からの英語チャットサポートサービス\n「kantan chat 」,「kantan chat」は最短3ヶ月からの契約期間で利用できる英語チャットサービスです。\...,"98,000円","100,000円",なし,なし,https://ai-products.net/product/kantan-chat/


In [114]:
len(COL_ELEMENTS)

7

# 出力したcsvファイルから順々にリンクをたどる

In [126]:
df = pd.read_csv("elements.csv").iloc[:,1:]
df.head()

visit_link_ = df["detail_link"]

In [137]:
goal = point1 = driver.find_elements_by_class_name("point")[0].text.replace("1\n", "")
goal

'トピック推定技術を競う国際コンテストで世界1位を獲得した、回答精度の高さを誇るパナソニック独自の開発エンジンを採用！'

In [146]:
driver = setup.driver_set()

/Users/ryusei/Library/Python/3.7/lib/python/site-packages/ipykernel_launcher.py:29: DeprecationWarning: use options instead of chrome_options


In [173]:
tar_ = []
goal_ = []
sca_ = []
p1_    = []
p2_    = []
p3_     = []

for i in range(len(visit_link_)):
    
    driver.get(visit_link_[i])
    time.sleep(0.5)
    
    # ない可能性がある
    try:
        target_ind = driver.find_element_by_class_name("industry-cate").text.replace(" ", ", ")
        goal = driver.find_element_by_class_name("purpose-cate").text.replace(" ", ", ")
        scale = driver.find_element_by_class_name("budget-cate").text.replace(" ", ", ")
    except:
        target_ind = "none"
        goal = "none"
        scale = "none"
    
    if len(driver.find_elements_by_class_name("point")) >=3:
        point1 = driver.find_elements_by_class_name("point")[0].text.replace("1\n", ", ")
        point2 = driver.find_elements_by_class_name("point")[1].text.replace("2\n", "")
        point3 = driver.find_elements_by_class_name("point")[2].text.replace("3\n", "")
    else:
        point1 = "none"
        point2 = "none"
        point3 = "none"
    
    tar_.append(target_ind)
    goal_.append(goal)
    sca_.append(scale)
    p1_.append(point1)
    p2_.append(point2)
    p3_.append(point3)
    print(f"{i+1} / {len(visit_link_)}を実行中")

1 / 39を実行中
2 / 39を実行中
3 / 39を実行中
4 / 39を実行中
5 / 39を実行中
6 / 39を実行中
7 / 39を実行中
8 / 39を実行中
9 / 39を実行中
10 / 39を実行中
11 / 39を実行中
12 / 39を実行中
13 / 39を実行中
14 / 39を実行中
15 / 39を実行中
16 / 39を実行中
17 / 39を実行中
18 / 39を実行中
19 / 39を実行中
20 / 39を実行中
21 / 39を実行中
22 / 39を実行中
23 / 39を実行中
24 / 39を実行中
25 / 39を実行中
26 / 39を実行中
27 / 39を実行中
28 / 39を実行中
29 / 39を実行中
30 / 39を実行中
31 / 39を実行中
32 / 39を実行中
33 / 39を実行中
34 / 39を実行中
35 / 39を実行中
36 / 39を実行中
37 / 39を実行中
38 / 39を実行中
39 / 39を実行中


In [174]:
detail_link_ = []
for i in visit_link_:
    detail_link_.append(i)

In [191]:
detail_pack = [  
                        detail_link_,
                        p1_,
                        p2_,
                        p3_,
                        tar_,
                        goal_,
                        sca_
                    ]

In [192]:
all_l = check_data_len(detail_pack)

all elements is same !


In [193]:
concat_detail = setup.auto_csv(detail_pack, COL_DETAILS, "detail", make=True)

In [194]:
concat_detail

,detail_link,point1,point2,point3,target_ind,goal,scale
0,https://ai-products.net/product/wistalk/,", トピック推定技術を競う国際コンテストで世界1位を獲得した、回答精度の高さを誇るパナソニッ...",WisTalkなら導入を手厚く支援。さまざまな企業の業務改革を成功に導いた専任SEによるサポ...,パナソニックグループの大規模運用で培った信頼性・活用実績のあるQ&Aテンプレートをご提供！Q...,全て,"問い合わせ対応, 業務改善（効率化）, コスト削減","大企業向け, 中小企業向け"
1,https://ai-products.net/product/hue/,", 日本最大規模の辞書＆NLP（自然言語処理）技術の応用で幅広い表現に対応",ユーザーは選択肢を選ぶだけ！AIが簡単・的確な対話をアシスト,利用状況がひと目で分かる！充実の管理者サポート機能で稼働後のPDCAも万全,"人材, 不動産, 金融, 物流, IT, 製造, 教育, 小売, 卸売, 保険, Webサー...","Webマーケ, 離脱防止, CVR改善, レコメンド, 問い合わせ対応, 受付, 業務改善（...","大企業向け, 中小企業向け"
2,https://ai-products.net/product/amie-helpbot/,", シナリオ作成不要、既存のドキュメントからチャットが作成",自動更新機能でFAQ更新はスケジュールで自動学習,テキストマイニングを活用することで高度な検索を実現,"人材, 不動産, 金融, 物流, IT, 製造, 教育, 小売, 卸売, 保険, Webサー...","SNSツール（LINE等）を用いた顧客問合せ対応, 従業員からの問合せ業務効率化, 顧客から...","大企業向け, 中小企業向け"
3,https://ai-products.net/product/aura/,", セグメント配信で反応率4倍\n顧客の属性や、LINE内での行動ログを元にした配信の出しわ...",botで情報回答や質問対応が可能\nbotをあらかじめ設定しておくことで、運用の工数をかける...,予約や決済もLINEで全て完結\nLINE内で予約から決済まで全て簡潔します。ユーザーにとっ...,"人材, 不動産, IT, 教育, 小売, 卸売, 保険, Webサービス, 通信, メディア...","LINEを用いたマーケティング, LINEを用いた顧客フォロー, LINEを用いた顧客管理","大企業向け, 中小企業向け"
4,https://ai-products.net/product/vareal-nlp/,", お客様のご要望を丁寧にヒアリングし、既存の製品の組み合わせや新たなAIモデルの作成など、...",NLPを使うことによりテキストに対して高精度な検索、レコメンデーションを実現します。,AIモデルの作成から実証、UIの作成、システムインテグレーションまでワンストップで対応します。,"人材, 不動産, 金融, 物流, IT, 製造, 教育, 小売, 卸売, 保険, Webサー...","レコメンド, 業務改善（効率化）","大企業向け, 中小企業向け, 個人事業主向け"
5,https://ai-products.net/product/schatti/,", 過去の問い合わせ内容を学習。\nAIが回答候補を評価が高い順に提示するので、迅速に回答が...",チャットボットとして利用可能。\n現場で質問・回答を学習または手動でデータセットすることで自...,"導入時・運用時の大幅なコスト削減。\n人的コストは97%、データ反映コストは500〜1,00...","人材, 不動産, 金融, 物流, IT, 製造, 教育, 小売, 卸売, 保険, Webサー...","Webマーケ, データ分析, 業務改善（効率化）, コスト削減, 接客, サイト運営, オン...","大企業向け, 中小企業向け"
6,https://ai-products.net/product/benefitter/,", 最適なものが選べる・つながる！\nボットの目的に応じて、Webブラウザ、Teams、Sl...",スモールスタートから拡大できる！\n業務を追加するときに、もう別々の製品を検討する必要はあり...,CTCのトータルサポート！\n“Benefitter”は、CTCが独自に開発したAIチャット...,"人材, 不動産, 金融, 物流, IT, 製造, 教育, 小売, 卸売, 保険, Webサー...","Webマーケ, 問い合わせ対応, 業務改善（効率化）, 接客, マーケティング, 営業支援,...",大企業向け
7,https://ai-products.net/product/digital-human/,", AIデジタルヒューマン（デジタル従業員）が24時間365日お客様の問い合わせに自動対応。...",チャットボットやNLP（自然言語処理）と連携が可能。既に運用されているサービスにアドオンする...,外見と声をコピーしたクローンキャラクターで、著名人やインフルエンサーをデジタル従業員として採...,"人材, 不動産, 金融, 物流, IT, 製造, 教育, 小売, 卸売, 保険, Webサー...","Webマーケ, 離脱防止, CVR改善, レコメンド, 問い合わせ対応, パーソナライズ, ...","大企業向け, 中小企業向け"
8,https://ai-products.net/product/hachidori/,", 7,000件を超える開発実績に基づいたシナリオ構築ノウハウ",経験豊富な専任チーム（カスタマーサクセス）の伴走サポート,高性能なAI会話エンジンで回答精度の継続的な向上,"人材, 不動産, IT, 教育, 小売, 保険, Webサービス, 通信, メディア, 官公庁","業務改善（効率化）, WEB、LINEマーケティング","大企業向け, 中小企業向け"
9,https://ai-products.net/product/kantan-chat/,", 導入コスト10万円、簡単なQAの準備だけですぐ開始できて最短3ヶ月の利用だけでもOK。業...",当社直営のフィリピンセンターで英語ネイティブスタッフが格安で対応。,AI搭載チャットが自己学習し4か月目以降は完全自動対応も可能、さらに運用コストを抑えられる。,"人材, 不動産, 金融, 物流, IT, 製造, 教育, 小売, 卸売, 保険, Webサー...",接客,"大企業向け, 中小企業向け"
